<a href="https://www.kaggle.com/code/averma111/autogluon-pss3e18?scriptVersionId=135200009" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture
!pip install autogluon


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os


import torch
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
from autogluon.common import space
import os.path


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import itertools

In [3]:
train_data = TabularDataset('/kaggle/input/playground-series-s3e18/train.csv')
subsample_size = 14838  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=42)
train_data.drop(columns=['EC3', 'EC4', 'EC5', 'EC6'],axis=1,inplace=True)
train_data.head()

Loaded data from: /kaggle/input/playground-series-s3e18/train.csv | Columns = 38 / 38 | Rows = 14838 -> 14838


,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2
11988,11988,118.000000,3.060660,2.536175,2.536175,1.640774,1.640774,0.548756,0.182919,17.721856,...,0.000000,0.000000,11.752550,12.841643,9.589074,33.833333,1,1,1,1
13039,13039,378.018438,10.685872,7.066210,7.066210,6.041834,6.041834,4.039540,2.503411,12.514062,...,0.000000,0.000000,23.468091,24.539800,13.825658,44.000000,0,0,0,1
13637,13637,277.172776,1.732051,0.682574,1.462072,0.886443,0.886443,0.087310,0.000000,24.415866,...,0.000000,0.000000,11.938611,19.262465,9.589074,44.333333,2,2,1,1
5537,5537,377.203227,11.265362,9.264697,9.264697,6.445092,6.445092,4.308487,2.706995,42.723899,...,0.000000,0.000000,17.744066,45.448667,28.250470,64.083333,0,0,0,0
8425,8425,315.695337,5.985416,3.473678,3.473678,2.634453,2.634453,1.507705,0.672861,0.000000,...,17.696186,6.066367,0.000000,6.103966,0.000000,37.166667,0,0,1,1


In [4]:
class MultilabelPredictor():
    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path, problem_types=None, eval_metrics=None, consider_labels_correlation=True,**kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        self.path = '/kaggle/working/'
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = self.eval_metrics[label]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i,**kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = '/kaggle/working/'
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [5]:
labels = ['EC1','EC2']  # which columns to predict based on the others
problem_types = ['multiclass','binary']  # type of each prediction problem
save_path = 'PSS3E18-predictEC1EC2'  # specifies folder to store trained models
time_limit =600

In [6]:
multi_predictor = MultilabelPredictor(labels=labels, 
                                      problem_types=problem_types, 
                                      path=save_path,
                                      eval_metrics=['accuracy','roc_auc_ovo_macro']
                                     )
multi_predictor.fit(train_data, time_limit=time_limit)

Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/kaggle/working/Predictor_EC1/"
AutoGluon Version:  0.8.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Jun 24 10:55:41 UTC 2023
Disk Space Avail:   20.94 GB / 20.96 GB (99.9%)
Train Data Rows:    14838
Train Data Columns: 32
Label Column: EC1
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Train Data Class Count: 2
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    32303.11 MB
	Train Data (Original)  Memory Usage: 3.8 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting 

Fitting TabularPredictor for label: EC1 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
	0.2s = Fit runtime
	32 features in original data used to generate 32 features in processed data.
	Train Data (Processed) Memory Usage: 3.8 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.26s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 13354, Val Rows: 1484
User-specified model hyperparameters to be fit:
{
	'NN_TORCH': {},
	'GBM': [{'extra_trees': True,

Fitting TabularPredictor for label: EC2 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  5 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2', 'EC1']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])       :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
		('int', ['bool']) :  1 | ['EC1']
	0.2s = Fit runtime
	33 features in original data used to generate 33 features in processed data.
	Train Data (Processed) Memory Usage: 3.81 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.23s ...
AutoGluon will gauge predictive performance using evaluation metric: 'roc_auc_ovo_macro'
	This metric expects predicted probabilities rather than predicted class labels, so you'll need to use predict_proba() instead of predict()
	To change this, specify the eval_metric parameter of Predictor()
Au

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('/kaggle/working/')


In [7]:
test_data = TabularDataset('/kaggle/input/playground-series-s3e18/test.csv')
test_data = test_data.sample(n=9893, random_state=42)
test_data.head()

Loaded data from: /kaggle/input/playground-series-s3e18/test.csv | Columns = 32 / 32 | Rows = 9893 -> 9893


,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
2807,17645,327.399591,4.414719,2.618262,2.618262,1.670820,1.670820,1.070620,0.694224,17.282269,...,0.000000,0.000000,18.199101,6.544756,10.902925,0.000000,0.000000,23.166667,0,0
5235,20073,292.492090,4.663902,2.767073,2.767073,1.904314,1.904314,1.060948,0.361460,5.969305,...,5.969305,0.000000,0.000000,6.076020,5.969305,0.000000,0.000000,37.666667,1,1
6291,21129,1122.912455,16.188780,10.684028,13.674919,8.012199,11.008637,7.199415,3.408511,89.249895,...,21.335138,0.000000,13.847474,18.181117,15.645394,69.410022,31.961948,73.715826,0,0
9789,24627,1727.925403,24.910940,15.637880,20.567315,11.722242,13.672761,11.312963,4.827185,90.658938,...,23.468091,32.533097,0.000000,37.099000,69.141353,63.436851,50.697492,119.375115,0,0
2868,17706,62.568425,2.642734,1.049739,1.049739,0.504904,0.504904,0.142577,0.000000,5.969305,...,28.148794,0.000000,0.000000,0.000000,5.969305,6.544756,4.794537,21.178241,1,1


In [8]:
multi_predictor = MultilabelPredictor.load(save_path)  
multi_predictor.evaluate(train_data)

Evaluating TabularPredictor for label: EC1 ...


Evaluation: accuracy on test data: 0.8531473244372557
Evaluations on test data:
{
    "accuracy": 0.8531473244372557,
    "balanced_accuracy": 0.7854774472817261,
    "mcc": 0.6691886841310939
}


Evaluating TabularPredictor for label: EC2 ...


Evaluation: roc_auc_ovo_macro on test data: 0.9174424649846953
Evaluations on test data:
{
    "roc_auc_ovo_macro": 0.9174424649846953,
    "accuracy": 0.7989621242755088,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "roc_auc": 0.9174424649846954,
    "f1": 0.8882478552429476,
    "precision": 0.7989621242755088,
    "recall": 1.0
}


{'EC1': {'accuracy': 0.8531473244372557,
  'balanced_accuracy': 0.7854774472817261,
  'mcc': 0.6691886841310939},
 'EC2': {'roc_auc_ovo_macro': 0.9174424649846953,
  'accuracy': 0.7989621242755088,
  'balanced_accuracy': 0.5,
  'mcc': 0.0,
  'roc_auc': 0.9174424649846954,
  'f1': 0.8882478552429476,
  'precision': 0.7989621242755088,
  'recall': 1.0}}

In [9]:
predictor_class = multi_predictor.get_predictor('EC1')
predictor_class.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.708221,0.520000,53.553873,0.006030,1.961087,2,True,14
1,CatBoost,0.703504,0.010880,3.594915,0.010880,3.594915,1,True,7
2,LightGBM,0.701482,0.010499,1.495715,0.010499,1.495715,1,True,4
3,XGBoost,0.701482,0.013940,2.026226,0.013940,2.026226,1,True,11
4,LightGBMXT,0.699461,0.009811,2.634305,0.009811,2.634305,1,True,3
5,NeuralNetFastAI,0.698787,0.042402,18.484606,0.042402,18.484606,1,True,10
6,NeuralNetTorch,0.698787,0.054208,10.422437,0.054208,10.422437,1,True,12
7,LightGBMLarge,0.697439,0.012823,3.538891,0.012823,3.538891,1,True,13
8,ExtraTreesEntr,0.696765,0.174112,2.809695,0.174112,2.809695,1,True,9
9,RandomForestEntr,0.692049,0.165132,8.579434,0.165132,8.579434,1,True,6


In [10]:
predictor_class = multi_predictor.get_predictor('EC2')
predictor_class.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.643585,0.266472,49.198582,0.002445,2.743248,2,True,14
1,LightGBMXT,0.632136,0.004603,2.189086,0.004603,2.189086,1,True,3
2,NeuralNetFastAI,0.631634,0.034998,20.153657,0.034998,20.153657,1,True,10
3,LightGBMLarge,0.629238,0.007066,3.928931,0.007066,3.928931,1,True,13
4,CatBoost,0.626671,0.005887,4.039607,0.005887,4.039607,1,True,7
5,XGBoost,0.625631,0.010430,2.694575,0.010430,2.694575,1,True,11
6,NeuralNetTorch,0.624820,0.053651,13.868644,0.053651,13.868644,1,True,12
7,LightGBM,0.624779,0.005893,2.154920,0.005893,2.154920,1,True,4
8,RandomForestGini,0.613745,0.163709,6.315016,0.163709,6.315016,1,True,5
9,ExtraTreesEntr,0.610345,0.168397,2.656244,0.168397,2.656244,1,True,9


In [11]:
predictions = multi_predictor.predict_proba(test_data)
print("Predictions:  \n", predictions)

Predicting with TabularPredictor for label: EC1 ...
Predicting with TabularPredictor for label: EC2 ...
Predictions:  
 {'EC1':              0         1
2807  0.241953  0.758047
5235  0.238493  0.761507
6291  0.365012  0.634988
9789  0.235141  0.764859
2868  0.196513  0.803487
...        ...       ...
5734  0.271537  0.728463
5191  0.238284  0.761716
5390  0.618955  0.381045
860   0.197483  0.802517
7270  0.505737  0.494263

[9893 rows x 2 columns], 'EC2':              0         1
2807  0.207643  0.792357
5235  0.228786  0.771214
6291  0.264231  0.735769
9789  0.229626  0.770374
2868  0.217526  0.782474
...        ...       ...
5734  0.248010  0.751990
5191  0.226419  0.773581
5390  0.126790  0.873210
860   0.202263  0.797737
7270  0.143192  0.856808

[9893 rows x 2 columns]}


In [12]:
class Submit:
    
    def submit_predictions(self,test_data):
        df_submit = pd.DataFrame(data={'id': test_data['id'],'EC1':list(predictions.values())[0][1],'EC2':list(predictions.values())[1][1]})
        df_submit.to_csv('submission.csv',index=False)
        print('Submission Completed!!')
        return df_submit
        
        
submit = Submit()
df_submit=submit.submit_predictions(test_data)

Submission Completed!!


In [13]:
df_submit

,id,EC1,EC2
2807,17645,0.758047,0.792357
5235,20073,0.761507,0.771214
6291,21129,0.634988,0.735769
9789,24627,0.764859,0.770374
2868,17706,0.803487,0.782474
...,...,...,...
5734,20572,0.728463,0.751990
5191,20029,0.761716,0.773581
5390,20228,0.381045,0.873210
860,15698,0.802517,0.797737
